# Music Recommender System using Apache Spark and Python
**Estimated time: 8hrs**

## Description

For this project, you are to create a recommender system that will recommend new musical artists to a user based on their listening history. Suggesting different songs or musical artists to a user is important to many music streaming services, such as Pandora and Spotify. In addition, this type of recommender system could also be used as a means of suggesting TV shows or movies to a user (e.g., Netflix). 

To create this system you will be using Spark and the collaborative filtering technique. The instructions for completing this project will be laid out entirely in this file. You will have to implement any missing code as well as answer any questions.

**Submission Instructions:** 
* Add all of your updates to this IPython file and do not clear any of the output you get from running your code.
* Upload this file onto moodle.

## Datasets

You will be using some publicly available song data from audioscrobbler, which can be found [here](http://www-etud.iro.umontreal.ca/~bergstrj/audioscrobbler_data.html). However, we modified the original data files so that the code will run in a reasonable time on a single machine. The reduced data files have been suffixed with `_small.txt` and contains only the information relevant to the top 50 most prolific users (highest artist play counts).

The original data file `user_artist_data.txt` contained about 141,000 unique users, and 1.6 million unique artists. About 24.2 million users’ plays of artists are recorded, along with their count.

Note that when plays are scribbled, the client application submits the name of the artist being played. This name could be misspelled or nonstandard, and this may only be detected later. For example, "The Smiths", "Smiths, The", and "the smiths" may appear as distinct artist IDs in the data set, even though they clearly refer to the same artist. So, the data set includes `artist_alias.txt`, which maps artist IDs that are known misspellings or variants to the canonical ID of that artist.

The `artist_data.txt` file then provides a map from the canonical artist ID to the name of the artist.

## Necessary Package Imports

In [1]:
from pyspark.mllib.recommendation import *
import random
from operator import *

## Loading data

Load the three datasets into RDDs and name them `artistData`, `artistAlias`, and `userArtistData`. View the README, or the files themselves, to see how this data is formated. Some of the files have tab delimeters while some have space delimiters. Make sure that your `userArtistData` RDD contains only the canonical artist IDs.

In [2]:
artistData=sc.textFile("/home/akshaynayak/01.Apache.Spark.Project-1.RecommenderSystems.FINAL/artist_data_small.txt")
artistAlias=sc.textFile("/home/akshaynayak/01.Apache.Spark.Project-1.RecommenderSystems.FINAL/artist_alias_small.txt")
userArtistData=sc.textFile("/home/akshaynayak/01.Apache.Spark.Project-1.RecommenderSystems.FINAL/user_artist_data_small.txt")


## Data Exploration

In the blank below, write some code that with find the users' total play counts. Find the three users with the highest number of total play counts (sum of all counters) and print the user ID, the total play count, and the mean play count (average number of times a user played an artist). Your output should look as follows:
```
User 1059637 has a total play count of 674412 and a mean play count of 1878.
User 2064012 has a total play count of 548427 and a mean play count of 9455.
User 2069337 has a total play count of 393515 and a mean play count of 1519.
```


In [3]:

alias_list=artistAlias.take(artistAlias.count())
#print len(alias_list)

alias_dict={}
for alias in alias_list:
    temp=alias.split("\t")
    if not temp[0] in alias_dict:
        alias_dict[temp[0]]=temp[1]

#print alias_list[1].split("\t")[0]

user_artist_data_list=userArtistData.take(userArtistData.count())
new_uadl=[]
for str in user_artist_data_list:
    spl=str.split(" ")
    if spl[1] in alias_dict:
        new_uadl.append((spl[0],alias_dict[spl[1]],int(spl[2])))
    else:
        new_uadl.append((spl[0],spl[1],int(spl[2])))    

userArtistData_tuple_rdd=sc.parallelize(new_uadl)
map1=userArtistData_tuple_rdd.map(lambda x: (x[0],x[2]))
total_play_count=map1.reduceByKey(lambda a,b:a+b)
#print total_play_count.takeOrdered(3,key=lambda x: -x[1])

#userArtistData=sc.parallelize(new_uadl)
#map2=userArtistData_tuple_rdd.map(lambda x: ((x[0],x[1]),x[2]))
#print map2.take(3)
#total_play_count2=map2.reduceByKey(lambda a,b:a+b)
#print total_play_count2.take(3)
#user_artist_count_map=total_play_count2.map(lambda x: (x[0][0],(x[1],1)))
#user_artist_count=user_artist_count_map.reduceByKey(lambda (a,b),(c,d):(a+c,b+d))
#print user_artist_count.takeOrdered(3,key=lambda x: -x[1][0])
#mean_values=user_artist_count.mapValues(lambda (a,b):a/b)
#print mean_values.take(3)
#joined_rdd=total_play_count.join(mean_values)
#print joined_rdd.take(3)

temp=total_play_count.takeOrdered(3,key=lambda x: -x[1])
#print joined_rdd.takeOrdered(3,key=lambda x: -x[1][0])

#print temp
count=[]
j=0
for i in temp:
    count.append([i[0],0])
    for tuple in new_uadl:
        if i[0]==tuple[0]:
            count[j][1]=count[j][1]+1
    j=j+1
        
#print count    
top3users_rdd=sc.parallelize(temp)
top3_users_artist_count_rdd=sc.parallelize(count)
join_list= top3users_rdd.join(top3_users_artist_count_rdd)

#print join_list.collect()
top_users_mean_list=join_list.map(lambda x: (x[0],x[1][0],x[1][0]/x[1][1]))
print_tuple=top_users_mean_list.collect()

#print print_tuple
sorted_print_tuple=sorted(print_tuple,key=lambda x: -x[1])
for i in sorted_print_tuple:
    print "User %s has a total play count of %d and a mean play count of %d"%(i[0],i[1],i[2])

    
  

#print temp[1][1][0]
#print new_uadl[0][0]
#print artistAlias.count()
#print artistData.count()
#print userArtistData.count()
#print artistAlias.take(3)
#print userArtistData.take(3)



User 1059637 has a total play count of 674412 and a mean play count of 1878
User 2064012 has a total play count of 548427 and a mean play count of 9455
User 2069337 has a total play count of 393515 and a mean play count of 1519


####  Splitting Data for Testing

Use the [randomSplit](http://spark.apache.org/docs/latest/api/python/pyspark.html#pyspark.RDD.randomSplit) function to divide the data (`userArtistData`) into:
* A training set, `trainData`, that will be used to train the model. This set should constitute 40% of the data.
* A validation set, `validationData`, used to perform parameter tuning. This set should constitute 40% of the data.
* A test set, `testData`, used for a final evaluation of the model. This set should constitute 20% of the data.

Use a random seed value of 13. Since these datasets will be repeatedly used you will probably want to persist them in memory using the [cache](http://spark.apache.org/docs/latest/api/python/pyspark.html#pyspark.RDD.cache) function.

In addition, print out the first 3 elements of each set as well as their sizes; if you created these sets correctly, your output should look as follows:
```
[(1059637, 1000049, 1), (1059637, 1000056, 1), (1059637, 1000113, 5)]
[(1059637, 1000010, 238), (1059637, 1000062, 11), (1059637, 1000112, 423)]
[(1059637, 1000094, 1), (1059637, 1000130, 19129), (1059637, 1000139, 4)]
19817
19633
10031
```

In [4]:
#print userArtistData.take(3)
userArtistData=userArtistData.map(lambda x:(int(x.split(" ")[0]),int(alias_dict[x.split(" ")[1]]),int(x.split(" ")[2])) if x.split(" ")[1]in alias_dict else (int(x.split(" ")[0]),int(x.split(" ")[1]),int(x.split(" ")[2])))
#print userArtistData.take(3)
trainData,validationData,testData=userArtistData.randomSplit([40,40,20],13)
print trainData.take(3)
print validationData.take(3)
print testData.take(3)
print trainData.count()
print validationData.count()
print testData.count()
#print userArtistData.take(3)

[(1059637, 1000049, 1), (1059637, 1000056, 1), (1059637, 1000113, 5)]
[(1059637, 1000010, 238), (1059637, 1000062, 11), (1059637, 1000112, 423)]
[(1059637, 1000094, 1), (1059637, 1000130, 19129), (1059637, 1000139, 4)]
19817
19633
10031


# The Recommender Model

For this project, we will train the model with implicit feedback. You can read more information about this from the collaborative filtering page: [http://spark.apache.org/docs/latest/mllib-collaborative-filtering.html](http://spark.apache.org/docs/latest/mllib-collaborative-filtering.html). The [function you will be using](http://spark.apache.org/docs/latest/api/python/pyspark.mllib.html#pyspark.mllib.recommendation.ALS.trainImplicit) has a few tunable parameters that will affect how the model is built. Therefore, to get the best model, we will do a small parameter sweep and choose the model that performs the best on the validation set

Therefore, we must first devise a way to evaluate models. Once we have a method for evaluation, we can run a parameter sweep, evaluate each combination of parameters on the validation data, and choose the optimal set of parameters. The parameters then can be used to make predictions on the test data.

### Model Evaluation

Although there may be several ways to evaluate a model, we will use a simple method here. Suppose we have a model and some dataset of *true* artist plays for a set of users. This model can be used to predict the top X artist recommendations for a user and these recommendations can be compared the artists that the user actually listened to (here, X will be the number of artists in the dataset of *true* artist plays). Then, the fraction of overlap between the top X predictions of the model and the X artists that the user actually listened to can be calculated. This process can be repeated for all users and an average value returned.

For example, suppose a model predicted [1,2,4,8] as the top X=4 artists for a user. Suppose, that user actually listened to the artists [1,3,7,8]. Then, for this user, the model would have a score of 2/4=0.5. To get the overall score, this would be performed for all users, with the average returned.

**NOTE: when using the model to predict the top-X artists for a user, do not include the artists listed with that user in the training data.**

Name your function `modelEval` and have it take a model (the output of ALS.trainImplicit) and a dataset as input. For parameter tuning, the dataset parameter should be set to the validation data (`validationData`). After parameter tuning, the model can be evaluated on the test data (`testData`).

In [5]:
uniqueArtist=list(set(userArtistData.map(lambda x: x[1]).collect()))
uniqueArtistrdd=sc.parallelize(uniqueArtist)
#print uniqueArtistrdd.count()
def modelEval(model,validationData):
    users=list(set(validationData.map(lambda x:x[0]).collect()))
    score={}
    #print len(users)
    for user in users:
        actualArtists=validationData.filter(lambda x:x[0]==user).map(lambda x:((x[0],x[1]),x[2]))
        toPredict=uniqueArtistrdd.map(lambda x: (user,x))
        predictions=model.predictAll(toPredict)
        artiststoberemoved=trainData.filter(lambda x:x[0]==user).collect()
        dict={}
        for tup in artiststoberemoved:
            dict[tup[1]]=1
        
        #actualArtists2=actualArtists.filter(lambda x:not x[0][1] in dict)    
        filteredPredictions=predictions.filter(lambda x:not x.product in dict).map(lambda x:((x.user,x.product),x.rating))
        topPredictions=filteredPredictions.takeOrdered(actualArtists.count(),key=lambda x:-x[1])
        topPredictionsrdd=sc.parallelize(topPredictions)
        common=actualArtists.join(topPredictionsrdd)
        score[user]=float(common.count())/actualArtists.count()
    total_score=0.000000
    for user in score.keys():
        total_score=total_score+score[user]
    #print score
    return total_score/len(score)
        
        
               
    

### Model Construction

Now we can build the best model possibly using the validation set of data and the `modelEval` function. Although, there are a few parameters we could optimize, for the sake of time, we will just try a few different values for the [rank parameter](http://spark.apache.org/docs/latest/mllib-collaborative-filtering.html#collaborative-filtering) (leave everything else at its default value, **except make `seed`=345**). Loop through the values [2, 10, 20] and figure out which one produces the highest scored based on your model evaluation function.

Note: this procedure may take several minutes to run.

For each rank value, print out the output of the `modelEval` function for that model. Your output should look as follows:
```
The model score for rank 2 is 0.090431
The model score for rank 10 is 0.095294
The model score for rank 20 is 0.090248
```

In [7]:
rank2model=ALS.trainImplicit(trainData,rank=2,seed=345)
print "The model score for rank 2 is %.6f"%modelEval(rank2model,validationData)
rank10model=ALS.trainImplicit(trainData,rank=10,seed=345)
print "The model score for rank 10 is %.6f"%modelEval(rank10model,validationData)
rank20model=ALS.trainImplicit(trainData,rank=20,seed=345)
print "The model score for rank 20 is %.6f"%modelEval(rank20model,validationData)

The model score for rank 2 is 0.090939
The model score for rank 10 is 0.095713
The model score for rank 20 is 0.090470


Now, using the bestModel, we will check the results over the test data. Your result should be ~`0.0507`.

In [10]:
bestModel = ALS.trainImplicit(trainData, rank=10, seed=345)
print "%.4f"%modelEval(bestModel, testData)

0.0512


## Trying Some Artist Recommendations
Using the best model above, predict the top 5 artists for user `1059637` using the [recommendProducts](http://spark.apache.org/docs/1.5.2/api/python/pyspark.mllib.html#pyspark.mllib.recommendation.MatrixFactorizationModel.recommendProducts) function. Map the results (integer IDs) into the real artist name using `artistAlias`. Print the results. The output should look as follows:
```
Artist 0: Brand New
Artist 1: Taking Back Sunday
Artist 2: Evanescence
Artist 3: Elliott Smith
Artist 4: blink-182
```

In [9]:
recommendedProducts=bestModel.recommendProducts(1059637,5)

#print artistData.take(3)
artistidnamedict={}
artistData2=artistData.map(lambda x:(int(x.split("\t")[0]),x.split("\t")[1]))
for artist in artistData2.collect():
    artistidnamedict[artist[0]]=artist[1]
artistlist=[]
for i in recommendedProducts:
    if i.product in alias_dict:
        artistlist.append(artistidnamedict[alias_dict[i.product]])
    else:
        artistlist.append(artistidnamedict[i.product])
for i in range(len(artistlist)):
    print "Artist %d: %s"%(i,artistlist[i])
    

Artist 0: Brand New
Artist 1: Taking Back Sunday
Artist 2: Evanescence
Artist 3: Elliott Smith
Artist 4: blink-182
